# The Pokemon Cookbook
This cookbook teaches you the concepts of the InfluxDB 3.0 Python Client library using a novel example of Pokemon data. The scenerio is to keep track of each trainer and the number of different pokemon they have caught.

<p align="center">
<img height="300" src="https://www.nicepng.com/png/full/62-622961_no-one-knows-if-people-eat-pokmon-png.png">
</p>

In [38]:

# Here we include all the imports required from influxdb_client_3 
import pandas as pd
import random
from pyarrow.flight import FlightDescriptor, FlightClient
from pyarrow import flight
# These are just some library imports for Plotly so we can make use of the interactive graphs.
import plotly.express as px
import plotly.io as pio

pio.renderers.default = "vscode"

## Writing Data
The first step is to write data into InfluxDB. We will use the `write_api` to write data into InfluxDB. In this example we are going to utilise `batching mode` to write data in batches. This is the most efficient way to write data into InfluxDB. To do this we are going to first setup some paramters for our client.

## Client Setup
Now that we have done the inital configurations of our write paramters its time to include these within our client initalization. The InfluxDB 3.0 Client can both write and query data. For now we will use it to write data based upon our configuration.

In [39]:
# In this example we are using the InfluxDBClient3 object to connect to the InfluxDB Cloud instance.
# We are also passing in the write_client_options we created earlier.
# The token, host and org are all required to connect to the InfluxDB Serverless instance.
# Note: that Org is optional with Dedicated instances.


client = FlightClient("grpc://localhost:8081")



now =  pd.Timestamp.now(tz='UTC').floor('ms')

# Lists of possible trainers
trainers = ["ash", "brock", "misty", "gary", "jessie", "james"]

# Read the CSV into a DataFrame. (Credit to @ritchie46 for the dataset)
pokemon_df = pd.read_csv("./pokemon.csv")

# Creating an empty list to store the data
data = []

# Dictionary to keep track of the number of times each trainer has caught each Pokémon
trainer_pokemon_counts = {}

# Number of entries we want to create
num_entries = 1000

### Generate some data
Now that we have our client setup lets start generating some data we can write to InfluxDB. Following the Pokemon example we will create a list of trainers and the number of pokemon they have caught. Trainers will catch pokemon randomly selected from our list stored within the Pandas DataFrame `pokemon_df`.

In [40]:
from IPython.display import display, HTML

# Generating random data
for i in range(num_entries):
    trainer = random.choice(trainers)
    
    # Randomly select a row from pokemon_df
    random_pokemon = pokemon_df.sample().iloc[0]
    caught = random_pokemon['Name']
    
    # Count the number of times this trainer has caught this Pokémon
    if (trainer, caught) in trainer_pokemon_counts:
        trainer_pokemon_counts[(trainer, caught)] += 1
    else:
        trainer_pokemon_counts[(trainer, caught)] = 1
    
    # Get the number for this combination of trainer and Pokémon
    num = trainer_pokemon_counts[(trainer, caught)]

    entry = {
        "trainer": trainer,
        "id": f"{0000 + random_pokemon['#']:04d}",
        "num": str(num),
        "name": caught,
        "level": random.randint(5, 20),
        "attack": random_pokemon['Attack'],
        "defense": random_pokemon['Defense'],
        "hp": random_pokemon['HP'],
        "speed": random_pokemon['Speed'],
        "type1": random_pokemon['Type 1'],
        "type2": random_pokemon['Type 2'],
        "timestamp": now
    }
    data.append(entry)

# Convert the list of dictionaries to a DataFrame
caught_pokemon_df = pd.DataFrame(data).set_index('timestamp')

# Print the DataFrame
display(caught_pokemon_df)

,trainer,id,num,name,level,attack,defense,hp,speed,type1,type2
timestamp,,,,,,,,,,,
2023-11-20 19:56:35.688000+00:00,ash,0043,1,Oddish,17,50,55,45,30,Grass,Poison
2023-11-20 19:56:35.688000+00:00,jessie,0060,1,Poliwag,15,50,40,40,90,Water,NaN
2023-11-20 19:56:35.688000+00:00,james,0024,1,Arbok,13,85,69,60,80,Poison,NaN
2023-11-20 19:56:35.688000+00:00,brock,0015,1,Beedrill,20,90,40,65,75,Bug,Poison
2023-11-20 19:56:35.688000+00:00,ash,0062,1,Poliwrath,16,95,95,90,70,Water,Fighting
...,...,...,...,...,...,...,...,...,...,...,...
2023-11-20 19:56:35.688000+00:00,ash,0015,1,Beedrill,9,90,40,65,75,Bug,Poison
2023-11-20 19:56:35.688000+00:00,jessie,0088,3,Grimer,14,80,50,80,25,Poison,NaN
2023-11-20 19:56:35.688000+00:00,ash,0028,1,Sandslash,9,100,110,75,65,Ground,NaN


### Write Data to InfluxDB
We will now write our newley created trainer data to InfluxDB. To do this we simply call client.write and pass in our dataframe. We then provide a static measurement name of `caught`. We also provide a series of tags to help identify our data. In this case we use the columns `['trainer', 'id', 'num']`. Note that we didn't provide our time column this is due to the fact we set this column as our `dataframe_index`. This means that the index column will be used as the time column.

In [41]:
import time
import pyarrow as pa

# Convert the DataFrame to an Arrow Table
table = pa.Table.from_pandas(caught_pokemon_df)

try:
    descriptor = FlightDescriptor.for_path("pokemon")
    writer, _ = client.do_put(descriptor, table.schema)
    writer.write_table(table)
    print(f"wrote: {table}")
    writer.close()
except Exception as e:
    print(f"Error writing point: {e}")



wrote: pyarrow.Table
trainer: string
id: string
num: string
name: string
level: int64
attack: int64
defense: int64
hp: int64
speed: int64
type1: string
type2: string
timestamp: timestamp[ns, tz=UTC]
----
trainer: [["ash","jessie","james","brock","ash",...,"ash","jessie","ash","james","jessie"]]
id: [["0043","0060","0024","0015","0062",...,"0015","0088","0028","0053","0002"]]
num: [["1","1","1","1","1",...,"1","3","1","2","3"]]
name: [["Oddish","Poliwag","Arbok","Beedrill","Poliwrath",...,"Beedrill","Grimer","Sandslash","Persian","Ivysaur"]]
level: [[17,15,13,20,16,...,9,14,9,16,14]]
attack: [[50,50,85,90,95,...,90,80,100,70,62]]
defense: [[55,40,69,40,95,...,40,50,110,60,63]]
hp: [[45,40,60,65,90,...,65,80,75,65,60]]
speed: [[30,90,80,75,70,...,75,25,65,115,60]]
type1: [["Grass","Water","Poison","Bug","Water",...,"Bug","Poison","Ground","Normal","Grass"]]
...


## Querying Data
We have now stored 1000 registered pokemon catches within InfluxDB. We can now query this data using the InfluxDB 3.0 Python Client to gain some insights into our data. We are going to use Plotly to visualise our data.

In [42]:

import json
from pyarrow.flight import Ticket



# Lets start with a simple query to understand our schema.
query = '''SHOW COLUMNS FROM pokemon'''

# We can use the query method to run a query against the database.
# Under the hood this creates a flight ticket and uses the FlightClient to run the query. 
# For this example we are using the pandas mode, which will return a pandas DataFrame.
# Language also allows us to specify the query language, in this case we are using SQL.
ticket_bytes = json.dumps({'sql':query, 'table':'pokemon'})
ticket = Ticket(ticket_bytes)
reader = client.do_get(ticket)
df = reader.read_all().to_pandas()

display(df)

,table_catalog,table_schema,table_name,column_name,data_type,is_nullable
0,datafusion,public,pokemon,trainer,Utf8,YES
1,datafusion,public,pokemon,id,Utf8,YES
2,datafusion,public,pokemon,num,Utf8,YES
3,datafusion,public,pokemon,name,Utf8,YES
4,datafusion,public,pokemon,level,Int64,YES
5,datafusion,public,pokemon,attack,Int64,YES
6,datafusion,public,pokemon,defense,Int64,YES
7,datafusion,public,pokemon,hp,Int64,YES
8,datafusion,public,pokemon,speed,Int64,YES
9,datafusion,public,pokemon,type1,Utf8,YES


### Simple InfluxQL Query
The first query we will run is a simple InfluxQL query to get the number of pokemon caught by each trainer. We will then use Plotly to visualise this data.

In [43]:
# Lets start with a simple query to understand our schema.
query = '''SELECT count("name") as count, trainer FROM pokemon GROUP BY trainer'''

ticket_bytes = json.dumps({'sql':query, 'table':'pokemon'})
ticket = Ticket(ticket_bytes)
reader = client.do_get(ticket)
df = reader.read_all().to_pandas()

fig1 = px.bar(df, x="trainer", y="count",color='trainer' ,title='Number of Pokémon caught in the last hour')
fig1.show()

In [44]:
# Lets start with a simple query to understand our schema.
query = '''SELECT count("name") as count, trainer, "type1" FROM pokemon GROUP BY "trainer","type1"'''

ticket_bytes = json.dumps({'sql':query, 'table':'pokemon'})
ticket = Ticket(ticket_bytes)
reader = client.do_get(ticket)
df = reader.read_all().to_pandas()


# We can use the query method to run a query against the database.
# Under the hood this creates a flight ticket and uses the FlightClient to run the query. 
# For this example we are using the pandas mode, which will return a pandas DataFrame.
# Language also allows us to specify the query language, in this case we are using SQL.

fig2 = px.bar(df, x="trainer", y="count", color='type1', barmode= 'group', title='Number of Pokémon caught in the last hour grouped by type')
fig2.show()

### Working with Arrow tables
So within the last section we discussed converting returned queries directly to Pandas Dataframes. However, we can also utilise their raw format, Arrow tables. Arrow tables are a columnar format that is more efficient for working with data.

Lets first start by adding more data with a random timestamp between now and 1 hour ago

In [45]:
num_entries = 10000 # You can reduce this if required. The more data the more interesting the results.
now =  pd.Timestamp.now(tz='UTC').floor('ms')
data = []


# Generating random data
for i in range(num_entries):
    # Randomise the timestamp
    timestamp = now - pd.Timedelta(minutes=random.randint(0, 60))
    trainer = random.choice(trainers)
    
    # Randomly select a row from pokemon_df
    random_pokemon = pokemon_df.sample().iloc[0]
    caught = random_pokemon['Name']
    
    # Count the number of times this trainer has caught this Pokémon
    if (trainer, caught) in trainer_pokemon_counts:
        trainer_pokemon_counts[(trainer, caught)] += 1
    else:
        trainer_pokemon_counts[(trainer, caught)] = 1
    
    # Get the number for this combination of trainer and Pokémon
    num = trainer_pokemon_counts[(trainer, caught)]

    entry = {
        "trainer": trainer,
        "id": f"{0000 + random_pokemon['#']:04d}",
        "num": str(num),
        "name": caught,
        "level": random.randint(5, 20),
        "attack": random_pokemon['Attack'],
        "defense": random_pokemon['Defense'],
        "hp": random_pokemon['HP'],
        "speed": random_pokemon['Speed'],
        "type1": random_pokemon['Type 1'],
        "type2": random_pokemon['Type 2'],
        "timestamp": timestamp
    }
    data.append(entry)

# Convert the list of dictionaries to a DataFrame
caught_pokemon_df = pd.DataFrame(data).set_index('timestamp')

# Print the DataFrame
display(caught_pokemon_df)
table = pa.Table.from_pandas(caught_pokemon_df)

try:
    descriptor = FlightDescriptor.for_path("caught")
    writer, _ = client.do_put(descriptor, table.schema)
    writer.write_table(table)
    print(f"wrote: {table}")
    writer.close()
except Exception as e:
    print(f"Error writing point: {e}")



KeyError: "None of ['time'] are in the columns"

Now lets return the result as an Arrow table. We can recreate he same aggregation we did on server side with the pyarrow library. the `mode='all'` parameter tells InfluxDB to return all data within the query result as a Arrow table. We can then use the `to_pandas()` method to convert the Arrow table to a Pandas DataFrame.

In [ ]:
query='''SELECT * FROM caught'''
ticket_bytes = json.dumps({'sql':query, 'table':'caught'})
ticket = Ticket(ticket_bytes)
reader = client.do_get(ticket)
table = reader.read_all()


# PyArrow Aggregation
aggregation = table.group_by(["trainer", "type1"]).aggregate([("name", "count")]).to_pandas()
fig3 = px.bar(aggregation, x="trainer", y="name_count", color='type1', barmode= 'group', title='Number of Pokémon caught grouped by type')
fig3.show()

### Saving to file
We can also save our query results to file. This is useful if we want to save our data for later use. We can save our data in a number of formats including CSV, JSON, Parquet and Apache Arrow. Lets save our data as a parquet file.

In [ ]:
import pyarrow.parquet as pq

# Write the table to a parquet file
pq.write_table(table, 'kanto.parquet')

### Working with Parquet files
This allows to show show off another feature of the InfluxDB 3.0 Python Client. We can parse our file directly to InfluxDB. This is useful if we want to upload data from a local file. Lets upload our parquet file to InfluxDB.

In [ ]:
# Read the parquet file into a table
table = pq.read_table('kanto.parquet')
descriptor = FlightDescriptor.for_path("caught")
writer, _ = client.do_put(descriptor, table.schema)
writer.write_table(table)
print(f"wrote: {table}")
writer.close()


wrote: pyarrow.Table
trainer: string
id: string
num: string
name: string
level: int64
attack: int64
defense: int64
hp: int64
speed: int64
type1: string
type2: string
timestamp: timestamp[ns, tz=UTC]
----
trainer: [["jessie","ash","gary","brock","misty",...,"brock","jessie","gary","misty","james"]]
id: [["0089","0053","0017","0019","0059",...,"0026","0123","0003","0052","0132"]]
num: [["1","1","1","1","1",...,"3","2","4","2","1"]]
name: [["Muk","Persian","Pidgeotto","Rattata","Arcanine",...,"Raichu","Scyther","VenusaurMega Venusaur","Meowth","Ditto"]]
level: [[16,18,19,9,13,...,11,16,12,11,14]]
attack: [[105,70,60,56,110,...,90,110,100,45,48]]
defense: [[75,60,55,35,80,...,55,80,123,35,48]]
hp: [[105,65,63,30,90,...,60,70,80,40,48]]
speed: [[50,115,71,72,95,...,110,105,80,90,48]]
type1: [["Poison","Normal","Normal","Normal","Fire",...,"Electric","Bug","Grass","Normal","Normal"]]
...


### Group by time query
Finally lets run a group by time query to get the number of pokemon caught by each trainer over the last hour grouped into 10 minute intervals. We will then use Plotly to visualise this data.

In [ ]:
query='''SELECT trainer,
  date_bin(INTERVAL '1 minute', timestamp, TIMESTAMP '1970-01-01 00:00:00Z') AS bin,
  count("name") AS count
FROM caught
GROUP BY bin, trainer
ORDER BY bin DESC'''

ticket_bytes = json.dumps({'sql':query, 'table':'caught'})
ticket = Ticket(ticket_bytes)
reader = client.do_get(ticket)
df = reader.read_all().to_pandas()
print(df)

fig4 = px.line(df, x="bin", y="count", color='trainer', title='Number of Pokémon caught in the last hour grouped by trainer and time')
fig4.show()

    trainer                   name                        timestamp
0    jessie                    Muk 2023-11-20 19:10:40.967000+00:00
1       ash                Persian 2023-11-20 19:10:40.967000+00:00
2      gary              Pidgeotto 2023-11-20 19:10:40.967000+00:00
3     brock                Rattata 2023-11-20 19:10:40.967000+00:00
4     misty               Arcanine 2023-11-20 19:10:40.967000+00:00
..      ...                    ...                              ...
995   brock                 Raichu 2023-11-20 19:10:40.967000+00:00
996  jessie                Scyther 2023-11-20 19:10:40.967000+00:00
997    gary  VenusaurMega Venusaur 2023-11-20 19:10:40.967000+00:00
998   misty                 Meowth 2023-11-20 19:10:40.967000+00:00
999   james                  Ditto 2023-11-20 19:10:40.967000+00:00

[1000 rows x 3 columns]


# Conclusion
We have now covered the basics of the InfluxDB 3.0 Python Client. I hope you found this novel cook book informative and fun. If you have any questions, bugs or feature requests please raise an issue on the [GitHub repo](https://github.com/InfluxCommunity/influxdb3-python/issues)


<p align="center">
<img height="100", width="100" src="https://i.pinimg.com/originals/18/15/44/181544facabe62d30c52e94b369f0f3a.png">
</p>